In [1]:
import torch 
from train import trainAR
from models import *
from matplotlib import pyplot as plt
import build
from tqdm.notebook import tqdm
import pytorch_lightning as pl
import torch.nn.functional as F

In [2]:
import build
loader = build.dataloader(data='dataset_v5', part=0.1, batch_size=10, shuffle=True)
print(len(loader))

10


In [3]:
width = 10
premodel = Sequential(
    ConvBlock(1, width, 1, shift=1),
    Res(GatedConvBlock(width, width, 2)),
    Res(GatedConvBlock(width, width, 4)),
    Res(GatedConvBlock(width, width, 8)),
    Res(GatedConvBlock(width, width, 16)),
    Res(GatedConvBlock(width, width, 32)),
    Res(GatedConvBlock(width, width, 64)),
    GatedConvBlock(width, 1, 128),
    torch.nn.Tanh(),
)

In [4]:
class DaNet(pl.LightningModule):
    def __init__(self, model_loader, noise=0.0):
        super().__init__()
        self.save_hyperparameters()
        self.model = eval(model_loader)
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, aux = batch
        x += torch.randn_like(x) * self.hparams.noise
        x_hat = self.forward(x)
        loss = F.mse_loss(x_hat, x)
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())
        return optimizer

In [7]:
model = DaNet(module_description(premodel), noise=0.3)
trainer = pl.Trainer(log_every_n_steps=5)
trainer.fit(model, train_dataloaders=loader)

c:\Users\msh24\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 2.7 K 
-------------------------------------
2.7 K     Trainable params
0         Non-trainable params
2.7 K     Total params
0.011     Total estimated model params size (MB)
c:\Users\msh24\AppData\Local\Programs\Python\Python39\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the 

Epoch 0:   0%|          | 0/10 [00:00<?, ?it/s] 

AttributeError: 'DaNet' object has no attribute 'noise'

In [ ]:
print(model)

DaNet(
  (model): Sequential(
    (0): Sequential(
      (0): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Activation(negative_slope=0.2)
      (2): Padded(
        padding=(2, -1)
        (f): Conv1d(1, 10, kernel_size=(2,), stride=(1,), bias=False)
      )
    )
    (1): Sum(
      (f): Identity()
      (g): Product(
        (f): Sequential(
          (0): Sequential(
            (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): Activation(negative_slope=0.2)
            (2): Padded(
              padding=(2, 0)
              (f): Conv1d(10, 10, kernel_size=(2,), stride=(1,), dilation=(2,), bias=False)
            )
          )
          (1): Tanh()
        )
        (g): Sequential(
          (0): Sequential(
            (0): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): Activation(negative_slope=0.2)
            (2): Padded(
           

In [ ]:
# trainAR(
#     trial=None,
#     device='cpu',
#     loader=loader,
#     noise=0.1,
#     model=model,
#     optim=torch.optim.Adam(model.parameters()),
#     sched=None,
#     criterion=build.criterion(),
#     logger=build.logger(console=True, save_rate=1e-3, sample_rate=0.5),
#     epochs=10
# )